In [ ]:
%pip install google-genai


In [5]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
import os
from google import genai
from google.genai import types


def generate():
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-2.0-flash"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text="""Total ban on agriculture in mekkah"""),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        thinking_config = types.ThinkingConfig(
        ),
        response_mime_type="application/json",
        response_schema=genai.types.Schema(
                        type = genai.types.Type.OBJECT,
                        required = ["region", "sector", "2020_impact", "2021_impact", "KPI"],
                        properties = {
                            "region": genai.types.Schema(
                                type = genai.types.Type.STRING,
                            ),
                            "sector": genai.types.Schema(
                                type = genai.types.Type.STRING,
                            ),
                            "2020_impact": genai.types.Schema(
                                type = genai.types.Type.NUMBER,
                            ),
                            "2021_impact": genai.types.Schema(
                                type = genai.types.Type.NUMBER,
                            ),
                            "KPI": genai.types.Schema(
                                type = genai.types.Type.STRING,
                            ),
                            "reasoning": genai.types.Schema(
                                type = genai.types.Type.STRING,
                            ),
                        },
                    ),
        system_instruction=[
            types.Part.from_text(text="""You are an expert macroeconomics analyst specializing in Saudi Arabia. Your task is to evaluate the impact of a given policy action on the key inputs of the Cobb-Douglas production function, which are population (labor), investments (capital), and productivity (total factor productivity).

For each policy action I provide, you must:

1.  **Identify the most directly impacted Cobb-Douglas input (KPI):** Choose ONE from \"population\", \"investments\", or \"productivity\" that the policy is most likely to influence.
2.  **Determine the affected region:** Select ONE region from \"Riyadh Region\" or \"Makkah Region\" where the impact is expected to be most significant.
3.  **Determine the affected sector:** Select ONE sector from \"A\" (Agriculture), \"B\" (Mining), or \"C\" (Manufacturing) where the impact is expected to be most significant. Use the ISIC classification.
4.  **Estimate the percentage impact for the years 2020 and 2021:** Based on your macroeconomic expertise and knowledge of Saudi Arabia, provide a percentage change (positive or negative) for the chosen KPI in the specified region and sector for each year. Express this as a decimal (e.g., a 2% increase is 0.02, a 5% decrease is -0.05). If you believe there will be no significant impact on any of the specified KPIs, regions, or sectors, return 0 for the impact values.
5.  **Justify your reasoning:** Briefly explain why you chose the specific KPI, region, sector, and the estimated impact percentages.

You MUST return your analysis in the following JSON format:

```json
{
  \"2020_impact\": [value],
  \"2021_impact\": [value],
  \"KPI\": \"[population|investments|productivity]\",
  \"region\": \"[Riyadh Region|Makkah Region]\",
  \"sector\": \"[A|B|C]\",
  \"reasoning\": \"[brief explanation]\"
}
Consider only the information provided in the policy action. Do not make assumptions beyond the scope of the described policy. Remember that extreme events, like a complete ban on a sector, should result in a -1.00 impact if productivity is chosen as the KPI.

Example 1:

Policy Action: The Saudi Arabian government announces a new initiative to provide low-interest loans specifically for the development of new agricultural technology and infrastructure within the Riyadh Region.

Expected LLM Output:

JSON

{
  \"2020_impact\": 0.02,
  \"2021_impact\": 0.04,
  \"KPI\": \"investments\",
  \"region\": \"Riyadh Region\",
  \"sector\": \"A\",
  \"reasoning\": \"The policy directly incentivizes capital investment in the agricultural sector of the Riyadh Region through low-interest loans. The impact is expected to be more pronounced in 2021 as projects initiated in 2020 come to fruition.\"
}
Example 2:

Policy Action: The government implements a strict new quota on the number of foreign workers allowed in the mining sector nationwide, effective immediately.

Expected LLM Output:

JSON

{
  \"2020_impact\": -0.03,
  \"2021_impact\": -0.05,
  \"KPI\": \"population\",
  \"region\": \"Riyadh Region\",
  \"sector\": \"B\",
  \"reasoning\": \"The quota directly reduces the labor available in the mining sector. While the policy is nationwide, the impact on population (labor input) is likely significant in key mining regions like Riyadh. The negative impact may slightly increase in 2021 as the effects of the quota fully materialize.\"
}"""),
        ],
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end="")

if __name__ == "__main__":
    generate()


{
  "2020_impact": -1.00,
  "2021_impact": -1.00,
  "KPI": "productivity",
  "region": "Makkah Region",
  "sector": "A",
  "reasoning": "A total ban on agriculture would completely eliminate agricultural output, resulting in a 100% decrease in productivity for that sector in the Makkah Region. The impact is immediate and sustained in both 2020 and 2021."
}